In [0]:
class Bronze():
    def __init__(self):
        self.base_data_dir = "/FileStore/tables"
        self.BOOTSTRAP_SERVER = "pkc-921jm.us-east-2.aws.confluent.cloud:9092"
        self.JAAS_MODULE = "org.apache.kafka.common.security.plain.PlainLoginModule"
        self.CLUSTER_API_KEY = "WQP32TVOEB3FYS7N"
        self.CLUSTER_API_SECRET = "wzWXLC/pnnOu+av+H1M56QdkESR4KiUItFUrYmKUNVN19lGGj0mmsrPebcDfmg8f"
    
    def getSchema(self):
        return """InvoiceNumber string, CreatedTime bigint, StoreID string, PosID string, CashierID string,
                CustomerType string, CustomerCardNo string, TotalAmount double, NumberOfItems bigint, 
                PaymentMethod string, TaxableAmount double, CGST double, SGST double, CESS double, 
                DeliveryType string,
                DeliveryAddress struct<AddressLine string, City string, ContactNumber string, PinCode string, 
                State string>,
                InvoiceLineItems array<struct<ItemCode string, ItemDescription string, 
                    ItemPrice double, ItemQty bigint, TotalValue double>>
            """

    def ingestFromKafka(self, startingTime = 1):
        return ( spark.readStream
            .format("kafka")
            .option("kafka.bootstrap.servers", self.BOOTSTRAP_SERVER)
            .option("kafka.security.protocol", "SASL_SSL")
            .option("kafka.sasl.mechanism", "PLAIN")
            .option("kafka.sasl.jaas.config", f"{self.JAAS_MODULE} required username='{self.CLUSTER_API_KEY}' password='{self.CLUSTER_API_SECRET}';")
            .option("subscribe", "Test")
            .option("maxOffsetsPerTrigger", 30)
            .option("startingTimestamp", startingTime)
            .load()
        )

    def getInvoices(self, kafka_df):
        from pyspark.sql.functions import from_json, cast
        return(
            kafka_df.select(kafka_df.key.cast('string').alias('key'),
                           from_json(kafka_df.value.cast('string'),self.getSchema()).alias("value"),
                           "topic","timestamp")
        )
    
    def upsert(self, invoices_df, batch_id):
        invoices_df.createOrReplaceTempView("invoices_df_temp_view")
        merge_stmt = """MERGE INTO invoices_bz s
                USING invoices_df_temp_view t
                ON s.value == t.value AND s.timestamp == t.timestamp
                WHEN MATCHED THEN
                UPDATE SET *
                WHEN NOT MATCHED THEN
                INSERT *
            """
        invoices_df._jdf.sparkSession().sql(merge_statement)

    def process(self, startingTime = 1):
        print(f"Starting Bronze Stream...", end='')
        invoicesDF = self.getInvoices(self.ingestFromKafka(startingTime))
        sQuery = (invoicesDF.writeStream
                  .queryName("bronze-ingestion")
                  .foreachBatch(self.upsert)
                  .option("checkpointLocation", f"{self.base_data_dir}/chekpoint/invoices_bz")
                  .outputMode("append")
                  .start()
        )
        print("Done")
        return sQuery
